*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you need to create a machine learning model that will classify SMS messages as either "ham" or "spam". A "ham" message is a normal message sent by a friend. A "spam" message is an advertisement or a message sent by a company.

You should create a function called `predict_message` that takes a message string as an argument and returns a list. The first element in the list should be a number between zero and one that indicates the likeliness of "ham" (0) or "spam" (1). The second element in the list should be the word "ham" or "spam", depending on which is most likely.

For this challenge, you will use the [SMS Spam Collection dataset](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/). The dataset has already been grouped into train data and test data.

The first two cells import the libraries and data. The final cell tests your model and function. Add your code in between these cells.


In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  #!pip install tf-nightly
  !pip install -q git+https://github.com/tensorflow/docs
# had to change as new tensorflow(2.7.0) does seem to work with kera yet so you can't call use keras functions
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
train_dataset = pd.read_csv(train_file_path, sep='\t', header=0, names=['type', 'message'])
test_dataset = pd.read_csv(test_file_path, sep='\t', header=0, names=['type', 'message'])

In [ ]:
train_dataset.head()

In [ ]:
clean = {"type": {"ham" : 0, "spam": 1}}
train_dataset = train_dataset.replace(clean)
test_dataset = test_dataset.replace(clean)

In [ ]:
train_dataset.tail()

In [ ]:
test_dataset.tail()

In [ ]:
train_labels = train_dataset.pop('type')
test_labels = test_dataset.pop('type')

In [ ]:
train_dataset.tail()

In [ ]:
train_messages = train_dataset['message'].tolist()
test_messages = test_dataset['message'].tolist()
print(train_messages)

In [ ]:
token = tf.keras.preprocessing.text.Tokenizer(num_words = 500, char_level=False, oov_token="<OOV>")
token.fit_on_texts(train_messages)
#token.fit_on_texts(test_message)

In [ ]:
word_index = token.word_index
word_index

In [ ]:
train_seq = token.texts_to_sequences(train_messages)
train_pad = tf.keras.preprocessing.sequence.pad_sequences(train_seq, maxlen=50, padding="post", truncating = "post")

test_seq = token.texts_to_sequences(test_messages)
test_pad = tf.keras.preprocessing.sequence.pad_sequences(test_seq, maxlen=50, padding="post", truncating = "post")

In [ ]:
print(test_pad.shape)
print(train_pad.shape)


In [ ]:
print(train_pad[0])

In [ ]:
model = keras.Sequential([
  keras.layers.Embedding(500, 16, input_length=50),
  keras.layers.GlobalAveragePooling1D(),
  keras.layers.Dense(24, activation='relu'),
  keras.layers.Dropout(0.2),
  keras.layers.Dense(1, activation='sigmoid')
])

model.summary()

In [ ]:
model.compile(loss='binary_crossentropy',optimizer='RMSprop' ,metrics=['accuracy'])

In [ ]:
history = model.fit(train_pad, train_labels, epochs=60, validation_data=(test_pad, test_labels), callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)])

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  new_text = token.texts_to_sequences([pred_text])
  new_pad = tf.keras.preprocessing.sequence.pad_sequences(new_text, maxlen=50, padding="post", truncating = "post")
  ans = model.predict(new_pad)
  print(ans)
  if(ans[0][0] > 0.5):
      prediction = [ans[0][0], 'spam']
  else:
      prediction = [ans[0][0], 'ham']

  return (prediction)

pred_text = "Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat..."

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
